In [9]:
import pandas as pd 
from reg_score import transferability_score, transferability_score_ridge
from logme import LogME
import pickle as pkl
import glob
import os
import numpy as np
from tqdm import tqdm
np.random.seed(42)

In [10]:
path = './asset/openmonkey_cub_features/*.pkl'
files = glob.glob(path)
len(files)

255

In [11]:
results = []
alphas = [0.0, 1.0]
for file in tqdm(files):
    result = {}
    with open(file, 'rb') as f:
        data = pkl.load(f)
        feature = data['train_features'][0]
        y_true = data['train_labels']
        n = y_true.shape[0]

    file_name = os.path.split(file)[-1]
    source = file_name.split('.')[0].split('_')[-3]
    target = file_name.split('.')[0].split('_')[-1]

    result['openmonkey_source'] = int(source)
    result['cub_target'] = int(target)
    
    for alpha in alphas:
        key = f"linmse_{alpha}"
        if alpha == 0.0:
            score = transferability_score(feature, y_true)
        else:
            score = transferability_score_ridge(feature, y_true, n * alpha)
        result[key] = score
    
    logme = LogME(regression=True)._fit_icml(feature, y_true)
    result['logme'] = logme

    results.append(result) 
results_df = pd.DataFrame(results)

100%|██████████| 255/255 [03:01<00:00,  1.40it/s]


In [12]:
results_df

,openmonkey_source,cub_target,linmse_0.0,linmse_1.0,logme
0,6,8,-0.067813,-0.080991,-0.130522
1,2,8,-0.066895,-0.080993,-0.121266
2,1,10,-0.012645,-0.017705,0.718397
3,13,10,-0.013624,-0.017714,0.654683
4,10,3,-0.030904,-0.039254,0.230165
...,...,...,...,...,...
250,9,7,-0.032198,-0.042293,0.218401
251,5,7,-0.032704,-0.042295,0.225340
252,13,5,-0.011901,-0.017452,0.676503
253,11,11,-0.038638,-0.058813,0.147090


In [13]:
mse_df = pd.read_csv("./asset/csv_file/ok_cub_head_mse.csv")
df = mse_df.merge(results_df, on=["openmonkey_source", "cub_target"])
df.head()

,openmonkey_source,cub_target,mse,linmse_0.0,linmse_1.0,logme
0,14,1,-0.087675,-0.035338,-0.043037,0.188655
1,14,2,-0.050266,-0.016480,-0.025843,0.542057
2,14,3,-0.075790,-0.029588,-0.039254,0.234948
3,14,4,-0.055507,-0.022277,-0.028812,0.404316
4,14,5,-0.034225,-0.011697,-0.017450,0.703516


In [14]:
keys = [
        'logme',
        'linmse_0.0', 
        'linmse_1.0',
        ]
for key in keys:
    df['mse'].corr(df[key])
    pearson_corr = df.mse.corr(df[key])
    kendall_corr = df.mse.corr(df[key], method='kendall')
    spearman_corr = df.mse.corr(df[key], method='spearman')
    print(f"Method: {key} - Pearson: {round(pearson_corr,3)} - Kendall: {round(kendall_corr,3)} - Spearman: {round(spearman_corr,3)}")

Method: logme - Pearson: 0.969 - Kendall: 0.904 - Spearman: 0.988
Method: linmse_0.0 - Pearson: 0.981 - Kendall: 0.901 - Spearman: 0.985
Method: linmse_1.0 - Pearson: 0.994 - Kendall: 0.924 - Spearman: 0.99
